In [14]:
import pandas as pd
import numpy as np
import os

In [15]:
paths = (
    'data\\run1\\hits-0.csv',
    'data\\run1\\hits-1.csv',
    'data\\dir1\\run2\\hits-2.csv',
    'data\\dir1\\run2\\hits-3.csv',
    'data\\dir1\\run3\\hits-4.csv',
    'data\\dir1\\run3\\hits-5.csv',
)

dfs = [pd.read_csv(path, skiprows=1) for path in paths]

In [16]:
for path, df in zip(paths, dfs):
    print(path, ':')
    print(df.head(), '\n')

data\run1\hits-0.csv :
   energy_deposit         x         y        z
0        0.017336 -0.118232  0.184559  74.1303
1        0.011640 -0.119106  0.181467  74.0223
2        0.044046 -0.121454  0.172552  73.7182
3        0.000552 -0.121539  0.172232  73.7073
4        0.026685 -0.123008  0.167092  73.5294 

data\run1\hits-1.csv :
   energy_deposit         x         y        z
0        0.008325 -0.327954  0.114894  74.1903
1        0.019345 -0.327982  0.114901  74.0849
2        0.030206 -0.328041  0.114915  73.8571
3        0.029599 -0.328100  0.114929  73.6300
4        0.019183 -0.328130  0.114936  73.5164 

data\dir1\run2\hits-2.csv :
   energy_deposit         x         y        z
0        0.000824 -0.114491  0.665390  74.2360
1        0.039282 -0.114926  0.665381  73.9151
2        0.026690 -0.115207  0.665386  73.6984
3        0.037438 -0.115610  0.665402  73.3754
4        0.019617 -0.115849  0.665414  73.1768 

data\dir1\run2\hits-3.csv :
   energy_deposit         x         y        z

In [17]:
# Geometry Dimensions
tubeMiddleZ = 8 / 2
# Histogram/Analysis Parameters
fullZlims = (-50, 80)
numZbins = 10 * (80 + 50)  # 10 per mm
zBins = np.linspace(fullZlims[0], fullZlims[1], numZbins + 1)

# Output directory for results (histogram sums, etc.).
outDirectory = os.path.normpath('data/testcalcs_output')

## Event Testing

In [18]:
path = paths[0]
df = dfs[0]
dfMiddle = df[df.z.abs() < tubeMiddleZ]

print(path)
print('Total energy deposit:', df.energy_deposit.sum())
print('Middle tube energy deposit:', dfMiddle.energy_deposit.sum())

data\run1\hits-0.csv
Total energy deposit: 6175.80174032489
Middle tube energy deposit: 72.950128318868


In [19]:
counts, bins = np.histogram(df.z, bins=zBins, weights=df.energy_deposit)
print(bins)
print(counts)
print(sum(counts) - df.energy_deposit.sum())  # Difference should be 0.
np.savetxt(os.path.join(outDirectory, 'event-hist.csv'), counts)

[-50.  -49.9 -49.8 ...  79.8  79.9  80. ]
[0. 0. 0. ... 0. 0. 0.]
1.4551915228366852e-11


## Run Testing

In [20]:
# More Histogram Parameters
xyLims = 7
numXYbins = 10 * 2 * xyLims  # 10 per mm
xyBins = np.linspace(-xyLims, xyLims, numXYbins + 1)
print(xyBins)

[-7.  -6.9 -6.8 -6.7 -6.6 -6.5 -6.4 -6.3 -6.2 -6.1 -6.  -5.9 -5.8 -5.7
 -5.6 -5.5 -5.4 -5.3 -5.2 -5.1 -5.  -4.9 -4.8 -4.7 -4.6 -4.5 -4.4 -4.3
 -4.2 -4.1 -4.  -3.9 -3.8 -3.7 -3.6 -3.5 -3.4 -3.3 -3.2 -3.1 -3.  -2.9
 -2.8 -2.7 -2.6 -2.5 -2.4 -2.3 -2.2 -2.1 -2.  -1.9 -1.8 -1.7 -1.6 -1.5
 -1.4 -1.3 -1.2 -1.1 -1.  -0.9 -0.8 -0.7 -0.6 -0.5 -0.4 -0.3 -0.2 -0.1
  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3
  1.4  1.5  1.6  1.7  1.8  1.9  2.   2.1  2.2  2.3  2.4  2.5  2.6  2.7
  2.8  2.9  3.   3.1  3.2  3.3  3.4  3.5  3.6  3.7  3.8  3.9  4.   4.1
  4.2  4.3  4.4  4.5  4.6  4.7  4.8  4.9  5.   5.1  5.2  5.3  5.4  5.5
  5.6  5.7  5.8  5.9  6.   6.1  6.2  6.3  6.4  6.5  6.6  6.7  6.8  6.9
  7. ]


In [23]:
eDeps, middleEdeps = [], []
zSums, xySums = [], []
for path, df in zip(paths[:2], dfs[:2]):
    print(path)
    # Energy sums.
    dfMiddle = df[df.z.abs() < tubeMiddleZ]
    edep = df.energy_deposit.sum()
    middleEdep = dfMiddle.energy_deposit.sum()
    eDeps.append(edep)
    middleEdeps.append(middleEdep)
    # Histograms.
    zSums.append(np.histogram(
        df.z, 
        bins=zBins, 
        weights=df.energy_deposit
    )[0])
    xySums.append(np.histogram2d(
        dfMiddle.x, 
        dfMiddle.y, 
        bins=2 * (xyBins,), 
        weights=dfMiddle.energy_deposit
    )[0])
    
# Energy sums.

eDeps, middleEdeps = np.array(eDeps), np.array(middleEdeps)
meanEdep, meanMiddleEdep = np.mean(eDeps), np.mean(middleEdeps)
eDepSigma = np.sqrt(np.mean((eDeps - meanEdep)**2))
middleEdepSigma = np.sqrt(np.mean((middleEdeps - meanMiddleEdep)**2))

print('Mean total energy deposit:', meanEdep)
print('Total energy deposit standard deviation:', eDepSigma)
print('Mean middle tube energy deposit:', meanMiddleEdep)
print('Middle tube energy deposit standard deviation:', middleEdepSigma)

# Histograms.
zSums, xySums = np.array(zSums), np.array(xySums)
zMeanSums, xyMeanSums = np.mean(zSums, axis=0), np.mean(xySums, axis=0)
zSumsSigmas, xySumsSigmas = np.std(zSums, axis=0), np.std(xySums, axis=0)
np.savetxt(os.path.join(outDirectory, 'run-zhist.csv'), zMeanSums)
np.savetxt(os.path.join(outDirectory, 'run-zhistsigmas.csv'), zSumsSigmas)
np.savetxt(os.path.join(outDirectory, 'run-xyhist.csv'), xyMeanSums)
np.savetxt(os.path.join(outDirectory, 'run-xyhistsigmas.csv'), xySumsSigmas.flatten())

data\run1\hits-0.csv
data\run1\hits-1.csv
Mean total energy deposit: 6306.383002036315
Total energy deposit standard deviation: 130.58126171142476
Mean middle tube energy deposit: 78.635272187729
Middle tube energy deposit standard deviation: 5.6851438688610045


array([ True,  True,  True, ...,  True,  True,  True])